In [6]:
# importing all necessary libraries
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import pandas as pd
import random

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
path = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/Deep Q Learning/nn_training_data_first_half.csv"
training_data_first_half = pd.read_csv(path)
path = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/Deep Q Learning/nn_training_data_second_half.csv"
training_data_second_half = pd.read_csv(path)
training_data = np.delete(pd.concat([training_data_first_half, training_data_second_half]).to_numpy(), [0], axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# print(len(training_data))

# training_data_list = []
# for row in training_data:
#   training_data_list.append(row.reshape(3,3))

# for i in range(5):
#   print(training_data_list[i])

In [13]:
class TicTacToe:
  def __init__(self):
    self.board = np.zeros((3, 3))

  def reset(self):
    self.board = np.zeros((3, 3))
    return self.board

  def check_win(self, player):
    # Check rows
    for i in range(3):
      if np.all(self.board[i, :] == player):
        return player

    # Check columns
    for j in range(3):
      if np.all(self.board[:, j] == player):
        return player

    #Check diagonal
    if np.all(np.diag(self.board) == player):
      return player
    if np.all(np.diag(np.fliplr(self.board)) == player):
      return player

    #Check tie
    if np.all(self.board != 0):
      return -1

    return 0

  # Check for empty places on board
  def possibilities(self):
    l = []

    for i in range(len(self.board)):
        for j in range(len(self.board)):

            if self.board[i][j] == 0:
                l.append((i, j))
    return(l)

  def random_place(self, player):
    selection = self.possibilities()
    current_loc = random.choice(selection)
    self.board[current_loc] = player

    return(self.board)

  def step(self, action, player):
    row, col = action
    self.board[row, col] = player
    done = self.check_win(player)
    reward = 0
    if done == 1 or done == 2:
      reward = 1
    return self.board, reward, done




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
episode_rewards = []
class PolicyNetworkTicTacToe(nn.Module):
    def __init__(self):
        super(PolicyNetworkTicTacToe, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(9, 128),
            nn.ReLU(),
            nn.Linear(128, 9),
            nn.Softmax(dim=-1),
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
#Tic Tac Toe version
def compute_discounted_rewards(rewards, gamma=0.99):
    discounted_rewards = []
    discounted_reward = 0
    i = 0
    for reward in reversed(rewards):
        discounted_reward = reward + gamma * discounted_reward
        discounted_rewards.insert(0, discounted_reward)
        #print(i," discounted_rewards: ", discounted_rewards)
        i += 1
    discounted_rewards = torch.tensor(discounted_rewards)
    #print("reversed discounted_rewards: ", discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-5)

    return discounted_rewards

#%%debug
debug = True

def train(env, policy, optimizer, episodes=1000):
  i = 0

  for episode in range(episodes):
    if debug:
      if i > 1:
        break
    state = env.reset()
    if debug:
      print(i," outer episode loop state: ", state)

    log_probs = []
    if debug:
      print(i, " outer episode loop log probabilities", log_probs)


    rewards = []
    if debug:
      print(i, "outer episode loop rewards: ", rewards)
    i += 1

    done = 0
    not_done_counter = 0
    while done == 0:
        state = torch.FloatTensor(state).unsqueeze(0)
        if debug:
          print("inner episode loop ", not_done_counter, " state: ", state)

        probs = policy(state)
        if debug:
          print("inner episode loop ", not_done_counter, " probs: ", probs)

        m = Categorical(probs)
        if debug:
          print("inner episode loop ", not_done_counter, " m: ", m)

        action = m.sample()
        if debug:
          print("inner episode loop ", not_done_counter, " action: ", action)
          print("")
          print("")
          print("")

        if debug:
          print("inner episode loop ", not_done_counter, "step input action.item(): ", action.item())
        state, reward, done, _ = env.step(action.item())
        if debug:
          print("inner episode loop after step ", not_done_counter, "step output state: ", state)
          print("inner episode loop after step ", not_done_counter, "step output reward: ", reward)
          print("inner episode loop after step ", not_done_counter, "step output done: ", done)
          print("")
          print("")
          print("")



        log_probs.append(m.log_prob(action))
        rewards.append(reward)

        not_done_counter += 1

        # Inside the train function, after an episode ends:
        if done:
            print("DONE!!!")
            episode_rewards.append(sum(rewards))
            discounted_rewards = compute_discounted_rewards(rewards)
            policy_loss = []
            for log_prob, Gt in zip(log_probs, disacounted_rewards):
                policy_loss.append(-log_prob * Gt)
            optimizer.zero_grad()
            policy_loss = torch.cat(policy_loss).sum()
            policy_loss.backward()
            optimizer.step()

            if episode % 50 == 0:
                print(f"Episode {episode}, Total Reward: {sum(rewards)}")
                print("")
                print("")
                print("")
            break
